<a href="https://colab.research.google.com/github/tsungchi-source/HW/blob/main/1022_week7_%E5%A4%9A%E6%A8%A1%E6%85%8B_Multimodal__%E8%9E%8D%E5%90%88%E6%96%B9%E5%BC%8F%E7%82%BAearly_%E5%8A%89%E5%AE%97%E6%97%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

gdown可以下載存放在Google雲端的檔案，類似wget

gdown github: https://github.com/wkentaro/gdown

In [ ]:
!pip install ipywidgets --upgrade
# 使用gdown下載資料
!gdown 1bZREL8xxNX4Us5vYtgsg6TomMTdaUARC # Reddit新聞資料
!gdown 1PLO60_bOaIYKzyoGyZHqUMv7WckvJBEw # 道瓊股價資料

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

# 資料集
* 文本資料：每日reddit新聞標題 - 代表市場情緒和輿論
* 股價資料：道瓊工業平均指數 - 代表市場實際表現

# 為什麼選擇這兩種資料？
1. 異質性：文本(非結構化) vs 數值(結構化) - 測試融合策略的效果
2. 時間序列性：都具有時間依賴性，適合LSTM處理
3. 相關性：新聞情緒理論上會影響股價，有內在關聯


資料集來源：https://www.kaggle.com/competitions/stock-market-prediction-and-sentimental-analysis/data

# 模型融合的各種型態
![多模態融合的各種形式](https://www.researchgate.net/publication/362028535/figure/fig2/AS:11431281126156761@1678559245845/llustration-of-early-fusion-late-fusion-and-middle-fusion-methods-used-by-multimodal.jpg)

本次作業主要實作的內容是(c)中期融合（Middle Fusion）

# 文本資料處理

## 目標：將新聞文本轉換為數值向量
新聞文本無法直接輸入機器學習模型，需要先轉換為數值表示


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

In [ ]:
import pandas as pd

# 讀取新聞資料
news_df = pd.read_csv('RedditNews(train).csv', encoding='utf-8')
# 資料清理：移除字串前後的 b'...' 標記
# 原因：原始資料中有些文本被存成 b'content' 格式，需要清理
# regex解釋：
#   ^b     - 匹配字串開頭的 'b'
#   ['\"]  - 匹配單引號或雙引號
#   |      - 或者
#   ['\"]$ - 匹配字串結尾的單引號或雙引號
news_df['News'] = news_df['News'].str.replace(r"^b['\"]|['\"]$", '', regex=True)
# 印出錢10筆資料
news_df.head(10)

In [ ]:
# 查看每日新聞數量
date_counts = news_df['Date'].value_counts().reset_index()
# 根據日期排序
date_counts.sort_values(by='Date', ascending=True)

In [ ]:
# 將每日新聞標題合併成一則新聞
# 目標：將同一天的多則新聞合併成單一文本
# 原因：模型需要每天一個輸入，而不是每天多個分散的新聞
news_dict = news_df.groupby('Date')['News'].apply(list).to_dict()
news_dict

In [ ]:
# 下載 Hugging Face Transformers
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
"""BERT文本編碼

為什麼使用BERT？
1. **語義理解**：BERT能理解詞語在上下文中的含義
2. **預訓練優勢**：已在大量文本上訓練，具備豐富語言知識
3. **向量化**：將文本轉換為768維的密集向量表示

技術細節
- 使用[CLS] token作為整體文本表示
- 每個新聞日期產生一個768維向量
- 這個向量包含了該日所有新聞的語義資訊
"""

# 初始化BERT模型和分詞器
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

# 設置為評估模式（關閉dropout等訓練相關功能）
# 原因：我們只用BERT做特徵提取，不進行訓練
bert_model.eval()
print('載入 BERT 成功')

In [ ]:
# 設置設備（CPU/GPU/MPS）偵測使用什麼設備
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
# 將模型移動到訓練設備
bert_model.to(device)
print(f'模型目前使用{device}')

In [ ]:
"""#批次文本編碼處理

逐日處理新聞文本，將每天的所有新聞合併後輸入BERT
"""
date_embeddings = {} # 儲存每日的文本嵌入向量




# 逐日期處理
for date, news_list in news_dict.items():

    # 步驟1合併當天的所有新聞為一個文本（用空格分隔）
    # 原因：需要將一天內的多則新聞統合成單一語義表示
    combined_text = " ".join(news_list)
    # 步驟2：文本預處理和分詞
    # 將文本 Tokenize 並輸入到 BERT
    # max_length=512：BERT的最大輸入長度限制
    # truncation=True：如果超過長度就截斷
    inputs = tokenizer(combined_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = bert_model(**inputs)
    print(f'{date} done.')

    # 提取[CLS] token的嵌入（位置0）作為整體文本表示
    # [CLS] token設計用來代表整個句子的語義
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).detach().cpu().numpy()

    # 儲存該日期的語義向量
    date_embeddings[date] = cls_embedding

In [ ]:
# 將嵌入字典轉換為DataFrame格式，方便後續處理
embedding_df = pd.DataFrame(date_embeddings).T  # 轉置：日期為行，特徵為列
embedding_df.columns = [f"Embedding_{i}" for i in range(embedding_df.shape[1])] # 重新命名列
embedding_df['Date'] = embedding_df.index
embedding_df['Date'] = pd.to_datetime(embedding_df['Date'], format='%Y-%m-%d') # 日期格式化
# 印出前10筆資料
embedding_df.head(10)

# 股價資料處理

# 目標：準備時間序列數值特徵
股價資料包含開盤價、收盤價、最高價、最低價、成交量等
我們主要使用收盤價作為預測目標的基準


In [ ]:
# 讀取股價資料
stock_data = pd.read_csv('DJIA_table(train).csv', encoding='utf-8')
# 日期欄位轉換成 datetime 格式
stock_data['Date'] = pd.to_datetime(stock_data['Date'], dayfirst=True)
# 設定日期為索引
stock_data.set_index('Date', inplace=True)
# 根據日期排序
stock_data = stock_data.sort_index()
# 印出前10筆
stock_data.head(10)

In [ ]:
"""
多模態資料合併
目標：將文本特徵和數值特徵按日期對齊
這是多模態學習的關鍵步驟：確保不同模態的資料在時間維度上對應
"""

# 只保留收盤價
price_df = stock_data[['Close']]
# 根據日期合併收盤價和嵌入
combined_df = pd.merge(price_df, embedding_df, on='Date')
# 設定日期為索引
combined_df.set_index('Date', inplace=True)
# 根據日期排序
combined_df = combined_df.sort_index()
# 印出前10筆
combined_df.head(10)

# 資料標記、滑動窗口

# 目標：創建預測標籤和時間序列窗口
1. **技術指標**：使用移動平均線判斷趨勢
2. **標籤策略**：短期均線>長期均線 = 多頭(1)，否則為空頭(0)
3. **滑動窗口**：將連續N天的資料作為一個訓練樣本

# 為什麼用移動平均交叉？
- **降噪**：平滑價格波動，減少雜訊
- **趨勢識別**：短長期均線交叉是經典的技術分析信號
- **實用性**：在實際交易中廣泛使用的策略



In [ ]:
import numpy as np

# 加入均線（後續標記漲跌用）
def add_MA(df, short_window=5, long_window=20):
    """
    添加移動平均線技術指標

    參數:
        df: 包含價格資料的DataFrame
        short_window: 短期移動平均窗口（預設5天）
        long_window: 長期移動平均窗口（預設20天）

    原理:
        - 短期均線反應近期價格變化
        - 長期均線反映整體趨勢
        - 兩者交叉點常被視為趨勢轉換信號
    """
    df['MA_short'] = df['Close'].rolling(window=short_window).mean()
    df['MA_long'] = df['Close'].rolling(window=long_window).mean()
    return df

# 標記漲跌
def create_labels(df):
    """
    根據移動平均線交叉創建分類標籤

    標籤邏輯:
        1 (多頭): 短期均線 > 長期均線，預期上漲
        0 (空頭): 短期均線 ≤ 長期均線，預期下跌

    這種標籤策略的優點:
        - 基於技術分析理論
        - 自動識別趨勢方向
        - 適合機器學習的二分類任務
    """
    # 計算移動平均線
    df = add_MA(df.copy())

    # 標記漲跌：短期均線在長期均線上方為1（多頭），下方為0（空頭）
    df['Target'] = (df['MA_short'] > df['MA_long']).astype(int)

    # 移除包含 NaN 的行（因為計算均線導致的）
    df = df.dropna()

    print("多頭信號（1）數量:", sum(df['Target'] == 1))
    print("空頭信號（0）數量:", sum(df['Target'] == 0))

    return df

# 準備序列資料
def prepare_sequence_data(df, sequence_length=5):
    """
    準備時間序列滑動窗口資料

    參數:
        df: 包含特徵和標籤的DataFrame
        sequence_length: 滑動窗口長度（預設5天）

    滑動窗口原理:
        - 使用過去N天的資料預測第N+1天
        - 例如：用第1-5天的資料預測第6天
        - 這樣可以讓模型學習時間序列的模式

    輸出:
        X_price: 價格時間序列 [樣本數, 時間步, 價格特徵數]
        X_news: 新聞時間序列 [樣本數, 時間步, 新聞特徵數]
        y: 對應的標籤 [樣本數]
    """
    # 分離特徵
    price_cols = ['Close', 'MA_short', 'MA_long']  # 價格相關特徵，現在包含均線
    news_cols = [col for col in df.columns if col.startswith('Embedding_')] # 新聞嵌入特徵

    X_price, X_news, y = [], [], []

    for i in range(len(df) - sequence_length):
        # 提取時間窗口內的價格序列
        # 價格序列（包含均線）
        price_seq = df[price_cols].iloc[i:i+sequence_length].values
        X_price.append(price_seq) # 添加至清單中
        # 提取時間窗口內的新聞序列
        # 新聞序列
        news_seq = df[news_cols].iloc[i:i+sequence_length].values
        X_news.append(news_seq) # 添加至清單中
        # 提取預測目標（窗口後一天的標籤）
        # 目標值
        y.append(df['Target'].iloc[i+sequence_length]) # 添加至清單中

    return np.array(X_price), np.array(X_news), np.array(y)

'''函數呼叫'''
# 加入均線
combined_df = add_MA(combined_df)

# 使用均線標記漲跌
combined_df = create_labels(combined_df)

# 創建序列資料
X_price, X_news, y = prepare_sequence_data(combined_df, sequence_length=5)

In [ ]:
"""
資料分割與標準化

目標：準備訓練和測試集
  1. 時間分割：按時間順序分割，避免未來資訊洩漏
  2. 標準化：統一特徵尺度，提升模型訓練效果
"""
# 切分訓練集和測試集(80%訓練，20%測試)
# 重要：按時間順序分割，不隨機打亂（避免時間洩漏）
split_index = int(len(X_price) * 0.8)

# 股價序列
X_price_train = X_price[:split_index]
X_price_test = X_price[split_index:]

# 新聞序列
X_news_train = X_news[:split_index]
X_news_test = X_news[split_index:]

# 目標值（0:跌, 1:漲）
y_train = y[:split_index]
y_test = y[split_index:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 標準化收盤價
price_scaler = MinMaxScaler()
X_price_train_scaled = price_scaler.fit_transform(X_price_train.reshape(-1, 1)).reshape(X_price_train.shape)
X_price_test_scaled = price_scaler.transform(X_price_test.reshape(-1, 1)).reshape(X_price_test.shape)

In [ ]:
# 顯示訓練資料集和測試資料集的大小 (資料筆數, 特徵數)
print('訓練集形狀：', X_price_train_scaled.shape, X_news_train.shape, y_train.shape)
print('測試集形狀：', X_price_test_scaled.shape, X_news_test.shape, y_test.shape)

# 建立模型-多模態融合模型建構

## 核心概念：不同的融合時機

### 1. **Early Fusion (早期融合)**
- **融合點**：輸入層
- **特點**：直接串接原始特徵
- **優勢**：結構簡單，計算效率高
- **劣勢**：可能造成特徵混雜，難以發揮各模態優勢

### 2. **Middle Fusion (中期融合)**
- **融合點**：特徵層
- **特點**：各模態先獨立處理，再融合特徵
- **優勢**：平衡獨立性與交互性
- **劣勢**：需要設計融合點

### 3. **Late Fusion (晚期融合)**
- **融合點**：決策層
- **特點**：各模態完全獨立預測，最後合併決策
- **優勢**：保持模態獨立性，可解釋性強
- **劣勢**：可能錯失深層交互機會

In [ ]:
""" 建立模型

## 作業重點：
**只需要修改下面的 fusion_type 參數，嘗試不同的融合方式！**

### 選擇融合方式：
- `fusion_type = "early"`   # 早期融合
- `fusion_type = "middle"`  # 中期融合 (base)
- `fusion_type = "late"`    # 晚期融合
"""

import tensorflow as tf # 引入tensorflow框架
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import (
    Input,  # 輸入層
    LSTM, # 長短期記憶
    Dense, # 全鏈接層
    Concatenate, # 合併層，這邊用來合併文本與數值特徵
    TimeDistributed, # 為了讓降維時不影響時間維度（窗口）
    Flatten, # 展平層
    Average, # 平均層
    Dropout # 隨機失活 - 防止過擬合
)

def build_multimodal_model(sequence_length, fusion_type="early"):
    """
    建立多模態融合模型

    參數:
        sequence_length: 時間序列長度（預設5天）
        fusion_type: 融合策略 ("early", "middle", "late")

    只需要修改fusion_type參數，就能體驗不同融合方式的效果
    """

    print(f"建立 {fusion_type.upper()} 融合模型...")

    # 輸入層定義（三種方式都相同）
    # price_input: [batch_size, sequence_length, 3] - 3個價格特徵
    price_input = Input(shape=(sequence_length, 3), name='price_input')
    # news_input: [batch_size, sequence_length, 768]
    news_input = Input(shape=(sequence_length, 768), name='news_input')

    if fusion_type == "early":
        # ========== 早期融合：輸入層直接合併 ==========
        print("早期融合：在輸入層直接串接特徵")

        # 步驟1：新聞特徵降維（避免維度不平衡）
        # 原因：新聞768維 vs 價格3維，維度差異太大，需要平衡
        # TimeDistributed：確保降維操作在每個時間步都執行
        news_reduced = TimeDistributed(Dense(32, activation='relu'),
                                     name='news_dim_reduction')(news_input)

        # 步驟2：特徵直接串接
        # 將價格[batch, 5, 3]和降維新聞[batch, 5, 32]串接成[batch, 5, 35]
        fused_features = Concatenate(axis=-1, name='early_fusion')([price_input, news_reduced])

        # 步驟3：統一LSTM處理混合特徵
        # 使用較大的LSTM(64)來處理複雜的混合特徵
        lstm_out = LSTM(64, name='unified_lstm')(fused_features)
        lstm_out = Dropout(0.2)(lstm_out)
        dense = Dense(32, activation='relu', name='dense_layer')(lstm_out)
        output = Dense(1, activation='sigmoid', name='output_layer')(dense)

    elif fusion_type == "middle":
        # ========== 中期融合：特徵層合併 ==========
        print("中期融合：分別處理後在特徵層合併")

        # 價格處理分支
        # 直接用LSTM處理價格時間序列，學習價格模式
        price_lstm = LSTM(32, name='price_lstm')(price_input)

        # 新聞處理分支
        # 先降維再用LSTM，學習新聞語義的時間演化
        news_dense = TimeDistributed(Dense(32), name='news_dense')(news_input)
        news_lstm = LSTM(16, name='news_lstm')(news_dense)

        # 中期融合點：在特徵表示層合併
        # 此時各模態已經學習到專門的特徵表示
        merged = Concatenate(name='middle_fusion')([price_lstm, news_lstm])
        # 最終決策層
        dense = Dense(32, activation='linear', name='dense_layer')(merged)
        output = Dense(1, activation='sigmoid', name='output_layer')(dense)

    elif fusion_type == "late":
        # ========== 晚期融合：決策層合併 ==========
        print("晚期融合：各自預測後合併結果")

        # 價格預測分支（完全獨立）
        price_lstm = LSTM(32, name='price_lstm')(price_input)
        price_dense = Dense(16, activation='relu', name='price_dense')(price_lstm)
        # 獨立的價格預測結果
        price_prediction = Dense(1, activation='sigmoid', name='price_prediction')(price_dense)

        # 新聞預測分支（完全獨立）
        news_dense = TimeDistributed(Dense(32), name='news_dense')(news_input)
        news_lstm = LSTM(16, name='news_lstm')(news_dense)
        news_dense2 = Dense(16, activation='relu', name='news_dense2')(news_lstm)
        # 獨立的新聞預測結果
        news_prediction = Dense(1, activation='sigmoid', name='news_prediction')(news_dense2)

        # 晚期融合點：合併兩個預測結果
        # 不是簡單平均，而是學習最佳組合權重
        predictions_concat = Concatenate(name='predictions_concat')([price_prediction, news_prediction])
        fusion_dense = Dense(8, activation='relu', name='fusion_dense')(predictions_concat)
        output = Dense(1, activation='sigmoid', name='output_layer')(fusion_dense)

    else:
        raise ValueError("fusion_type 必須是 'early', 'middle', 或 'late'")

    # 建立模型
    model = Model(inputs=[price_input, news_input], outputs=output,
                  name=f'{fusion_type}_fusion_model')

    # 編譯模型
    model.compile(optimizer='adam', # Adam優化器：自適應學習率，適合大多數任務
                 loss='binary_crossentropy', # 二元交叉熵：適合二分類任務
                 metrics=['accuracy']) # 評估指標：準確率

    print(f"{fusion_type.upper()} 融合模型建立完成！")
    print(f"總參數量: {model.count_params():,}")

    return model

In [ ]:
# 作業：修改這裡的 fusion_type 參數！
# ============================================
fusion_type = "early"  # ← 在這裡修改：early, middle, late
# ============================================

multimodal = build_multimodal_model(sequence_length=5, fusion_type=fusion_type)
multimodal.summary()

plot_model(
  multimodal, # 模型
  to_file=f'./{fusion_type}_fusion_model.png', # 可視化圖儲存路徑
  show_shapes=False,  # 顯示維度
  show_layer_names=True, # 顯示模型名稱
  dpi=120 # 解析度
  )

# 顯示圖片（在 Colab 中）
from IPython.display import Image, display
print(f"{fusion_type.upper()} 融合模型架構圖：")
display(Image(f'./{fusion_type}_fusion_model.png'))

In [ ]:
"""## 訓練模型"""

print(f"開始訓練 {fusion_type.upper()} 融合模型...")
start_time = time.time()

history = multimodal.fit(
    [X_price_train_scaled, X_news_train], # X (股價, 新聞)，順序需對應模型定義時的結構
    y_train, # y (漲或跌)
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    verbose=1
)

training_time = time.time() - start_time
print(f"訓練時間: {training_time:.1f} 秒")

# 結果評估

In [ ]:
"""# 結果評估"""

# 測試集評估
loss, accuracy = multimodal.evaluate([X_price_test_scaled, X_news_test], y_test, verbose=0)

print("="*50)
print(f"{fusion_type.upper()} 融合模型評估結果")
print("="*50)
print(f"測試準確率: {accuracy:.4f}")
print(f"參數量: {multimodal.count_params():,}")
print(f"訓練時間: {training_time:.1f} 秒")
print("="*50)